## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'C:/Users/jekim/OneDrive/바탕 화면/m5-forecasting-accuracy/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'

In [3]:
####################################################################################
################## 2-3. nonrecursive model by store & dept #########################
####################################################################################

In [4]:
cvs = ['private']

In [5]:
STORES = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
DEPTS = ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']
CLUSTERS = [0,1,2,3,4,5,6,7,8,9]

In [6]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [7]:
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [8]:
FIRST_DAY = 710
remove_feature = ['id',
                  'state_id',
                  'store_id',
                  'dept_id',
                  'cat_id',
                  'tskm_5', 'tskm_7', 'tskm_30', 'tskm_50',
                  'date','wm_yr_wk','d','sales']

cat_var = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
cat_var = list(set(cat_var) - set(remove_feature))

In [9]:
grid2_colnm = ['sell_price', 'price_max', 'price_min', 'price_std',
               'price_mean', 'price_norm', 'price_nunique', 'item_nunique',
               'price_momentum', 'price_momentum_m', 'price_momentum_y']

grid3_colnm = ['event_name_1', 'event_type_1', 'event_name_2',
               'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m',
               'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end']

lag_colnm = [ 'sales_lag_28', 'sales_lag_29', 'sales_lag_30',
             'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34',
             'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38',
             'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42',
             
             'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14',
             'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60',
             'rolling_std_60', 'rolling_mean_180', 'rolling_std_180']

mean_enc_colnm = [
    
    'enc_item_id_store_id_mean', 'enc_item_id_store_id_std'

]

In [10]:
########################### Make grid
#################################################################################
def prepare_data(cluster):
    
    grid_1 = pd.read_pickle(processed_data_dir+"grid_part_1.pkl")
    grid_2 = pd.read_pickle(processed_data_dir+"grid_part_2.pkl")[grid2_colnm]
    grid_3 = pd.read_pickle(processed_data_dir+"grid_part_3.pkl")[grid3_colnm]

    grid_df = pd.concat([grid_1, grid_2, grid_3], axis=1)
    del grid_1, grid_2, grid_3; gc.collect()
    
    grid_df = grid_df[grid_df['tskm_10'] == cluster]
    grid_df = grid_df[grid_df['d'] >= FIRST_DAY]
    
    lag = pd.read_pickle(processed_data_dir+"lags_df_28.pkl")[lag_colnm]
    
    lag = lag[lag.index.isin(grid_df.index)]
    
    grid_df = pd.concat([grid_df,lag], axis=1)
    
    del lag; gc.collect()
    

    mean_enc = pd.read_pickle(processed_data_dir+"mean_encoding_df.pkl")[mean_enc_colnm]
    mean_enc = mean_enc[mean_enc.index.isin(grid_df.index)]
    
    grid_df = pd.concat([grid_df,mean_enc], axis=1)    
    del mean_enc; gc.collect()
    
    grid_df = reduce_mem_usage(grid_df)
    
    
    
    return grid_df

In [11]:
validation = {
    'cv1' : [1551, 1610],
    'cv2' : [1829,1857],
    'cv3' : [1857, 1885],
    'cv4' : [1885,1913],
    'public' : [1913, 1941],
    'private' : [1941, 1969]
}

### cv1 : 2015-04-28 ~ 2015-06-26

### cv2 : 2016-02-01 ~ 2016-02-28

### cv3 : 2016-02-29 ~ 2016-03-27

### cv4 : 2016-03-28 ~ 2016-04-24

In [12]:
########################### Model params
#################################################################################
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1,
                    'seed' : 1995
                } 

In [13]:
########################### Train Models
#################################################################################

rmsse_bycv = dict()

for cv in cvs:
    print('cv : day', validation[cv])

    pred_list = []
    for cluster in CLUSTERS:

        print('CLUSTER ', str(cluster), 'start')
        grid_df = prepare_data(cluster)

        model_var = grid_df.columns[~grid_df.columns.isin(remove_feature)]

        tr_mask = (grid_df['d'] <= validation[cv][0]) & (grid_df['d'] >= FIRST_DAY)
        vl_mask = (grid_df['d'] > validation[cv][0]) & (grid_df['d'] <= validation[cv][1])

        train_data = lgb.Dataset(grid_df[tr_mask][model_var], 
                                 label=grid_df[tr_mask]['sales'])

        valid_data = lgb.Dataset(grid_df[vl_mask][model_var], 
                                 label=grid_df[vl_mask]['sales'])

        m_lgb = lgb.train(lgb_params, train_data, valid_sets = [valid_data, train_data], verbose_eval=100) 
        model_name = model_dir+'non_recur_model_storedept_cluster10_'+str(cluster)+'.bin'
        pickle.dump(m_lgb, open(model_name, 'wb'))
            
        display(pd.DataFrame({'name':m_lgb.feature_name(),
                              'imp':m_lgb.feature_importance()}).sort_values('imp',ascending=False).head(25))
            
        del grid_df, train_data, valid_data, m_lgb, tr_mask, vl_mask; gc.collect

cv : day [1941, 1969]
CLUSTER  0 start
[100]	training's rmse: 0.807102	valid_0's rmse: 0.579144
[200]	training's rmse: 0.784379	valid_0's rmse: 0.573194
[300]	training's rmse: 0.775152	valid_0's rmse: 0.588379
[400]	training's rmse: 0.759892	valid_0's rmse: 0.602599
[500]	training's rmse: 0.749144	valid_0's rmse: 0.61075
[600]	training's rmse: 0.739452	valid_0's rmse: 0.615864
[700]	training's rmse: 0.732137	valid_0's rmse: 0.617847
[800]	training's rmse: 0.727122	valid_0's rmse: 0.619196
[900]	training's rmse: 0.722033	valid_0's rmse: 0.619196
[1000]	training's rmse: 0.717721	valid_0's rmse: 0.619457
[1100]	training's rmse: 0.714093	valid_0's rmse: 0.621703
[1200]	training's rmse: 0.710958	valid_0's rmse: 0.622202
[1300]	training's rmse: 0.707217	valid_0's rmse: 0.624139
[1400]	training's rmse: 0.704702	valid_0's rmse: 0.624303
[1500]	training's rmse: 0.702025	valid_0's rmse: 0.624209
[1600]	training's rmse: 0.699139	valid_0's rmse: 0.624656
[1700]	training's rmse: 0.696273	valid_0's 

,name,imp
0,item_id,135375
22,tm_w,36581
51,rolling_mean_180,29724
14,event_name_1,28845
49,rolling_mean_60,26090
52,rolling_std_180,25637
12,price_momentum_m,23466
50,rolling_std_60,22574
47,rolling_mean_30,21633
48,rolling_std_30,21396


CLUSTER  1 start
[100]	training's rmse: 3.2569	valid_0's rmse: 3.20828
[200]	training's rmse: 2.92024	valid_0's rmse: 4.14566
[300]	training's rmse: 2.81693	valid_0's rmse: 4.4233
[400]	training's rmse: 2.75268	valid_0's rmse: 4.53628
[500]	training's rmse: 2.70317	valid_0's rmse: 4.56869
[600]	training's rmse: 2.66897	valid_0's rmse: 4.57369
[700]	training's rmse: 2.63747	valid_0's rmse: 4.57361
[800]	training's rmse: 2.61026	valid_0's rmse: 4.56774
[900]	training's rmse: 2.58702	valid_0's rmse: 4.5744
[1000]	training's rmse: 2.5645	valid_0's rmse: 4.57641
[1100]	training's rmse: 2.54488	valid_0's rmse: 4.57303
[1200]	training's rmse: 2.52566	valid_0's rmse: 4.56667
[1300]	training's rmse: 2.50758	valid_0's rmse: 4.56004
[1400]	training's rmse: 2.49113	valid_0's rmse: 4.55531
[1500]	training's rmse: 2.47385	valid_0's rmse: 4.54737
[1600]	training's rmse: 2.45826	valid_0's rmse: 4.54134
[1700]	training's rmse: 2.44243	valid_0's rmse: 4.53614
[1800]	training's rmse: 2.4279	valid_0's rms

,name,imp
0,item_id,144280
22,tm_w,39104
14,event_name_1,25729
51,rolling_mean_180,25262
12,price_momentum_m,25016
52,rolling_std_180,24098
50,rolling_std_60,21939
49,rolling_mean_60,21934
13,price_momentum_y,21174
48,rolling_std_30,18874


CLUSTER  2 start
[100]	training's rmse: 1.87442	valid_0's rmse: 1.69118
[200]	training's rmse: 1.79689	valid_0's rmse: 2.01455
[300]	training's rmse: 1.76739	valid_0's rmse: 2.12474
[400]	training's rmse: 1.74422	valid_0's rmse: 2.16659
[500]	training's rmse: 1.72574	valid_0's rmse: 2.1841
[600]	training's rmse: 1.71042	valid_0's rmse: 2.18961
[700]	training's rmse: 1.69615	valid_0's rmse: 2.19631
[800]	training's rmse: 1.68376	valid_0's rmse: 2.20067
[900]	training's rmse: 1.67224	valid_0's rmse: 2.20425
[1000]	training's rmse: 1.66136	valid_0's rmse: 2.20453
[1100]	training's rmse: 1.65032	valid_0's rmse: 2.2095
[1200]	training's rmse: 1.64052	valid_0's rmse: 2.21116
[1300]	training's rmse: 1.63087	valid_0's rmse: 2.21182
[1400]	training's rmse: 1.62128	valid_0's rmse: 2.21302
[1500]	training's rmse: 1.61203	valid_0's rmse: 2.21341
[1600]	training's rmse: 1.60313	valid_0's rmse: 2.21465
[1700]	training's rmse: 1.59493	valid_0's rmse: 2.21423
[1800]	training's rmse: 1.58687	valid_0's 

...

,name,imp
0,item_id,147205
22,tm_w,28966
14,event_name_1,27250
51,rolling_mean_180,23159
12,price_momentum_m,22093
44,rolling_std_7,20313
49,rolling_mean_60,19302
52,rolling_std_180,19068
47,rolling_mean_30,18644
45,rolling_mean_14,18640


CLUSTER  8 start
[100]	training's rmse: 3.32603	valid_0's rmse: 4.13598
[200]	training's rmse: 2.96067	valid_0's rmse: 5.56603
[300]	training's rmse: 2.7964	valid_0's rmse: 5.91232
[400]	training's rmse: 2.67451	valid_0's rmse: 6.037
[500]	training's rmse: 2.5843	valid_0's rmse: 6.05408
[600]	training's rmse: 2.51008	valid_0's rmse: 6.09643
[700]	training's rmse: 2.44115	valid_0's rmse: 6.09874
[800]	training's rmse: 2.38054	valid_0's rmse: 6.11908
[900]	training's rmse: 2.32509	valid_0's rmse: 6.10445
[1000]	training's rmse: 2.27891	valid_0's rmse: 6.10281
[1100]	training's rmse: 2.23626	valid_0's rmse: 6.10041
[1200]	training's rmse: 2.19525	valid_0's rmse: 6.11552
[1300]	training's rmse: 2.15639	valid_0's rmse: 6.14557
[1400]	training's rmse: 2.12278	valid_0's rmse: 6.14355
[1500]	training's rmse: 2.09104	valid_0's rmse: 6.15094
[1600]	training's rmse: 2.06256	valid_0's rmse: 6.13919
[1700]	training's rmse: 2.03523	valid_0's rmse: 6.13322
[1800]	training's rmse: 2.00791	valid_0's rm

,name,imp
0,item_id,116271
22,tm_w,35785
51,rolling_mean_180,30164
52,rolling_std_180,26122
12,price_momentum_m,25233
50,rolling_std_60,24772
49,rolling_mean_60,24467
21,tm_d,22092
48,rolling_std_30,21610
44,rolling_std_7,21252


CLUSTER  9 start
[100]	training's rmse: 2.33684	valid_0's rmse: 2.1063
[200]	training's rmse: 2.21227	valid_0's rmse: 2.56285
[300]	training's rmse: 2.17671	valid_0's rmse: 2.69316
[400]	training's rmse: 2.15034	valid_0's rmse: 2.74227
[500]	training's rmse: 2.12728	valid_0's rmse: 2.76566
[600]	training's rmse: 2.10746	valid_0's rmse: 2.77764
[700]	training's rmse: 2.08891	valid_0's rmse: 2.78667
[800]	training's rmse: 2.07199	valid_0's rmse: 2.78894
[900]	training's rmse: 2.05774	valid_0's rmse: 2.79217
[1000]	training's rmse: 2.04437	valid_0's rmse: 2.79557
[1100]	training's rmse: 2.03105	valid_0's rmse: 2.79879
[1200]	training's rmse: 2.01808	valid_0's rmse: 2.80061
[1300]	training's rmse: 2.00601	valid_0's rmse: 2.80075
[1400]	training's rmse: 1.99529	valid_0's rmse: 2.80294
[1500]	training's rmse: 1.98557	valid_0's rmse: 2.80557
[1600]	training's rmse: 1.97642	valid_0's rmse: 2.80473
[1700]	training's rmse: 1.9677	valid_0's rmse: 2.80398
[1800]	training's rmse: 1.95905	valid_0's 

,name,imp
0,item_id,159730
22,tm_w,38517
14,event_name_1,27852
12,price_momentum_m,25064
51,rolling_mean_180,24143
52,rolling_std_180,21125
49,rolling_mean_60,20487
50,rolling_std_60,18510
10,item_nunique,17851
23,tm_m,16943
